# **Tiempo estimado de venta**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre or 'df_junio_2023_queretaro' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'tiempo_de_publicacion':'antiguedad','Categoria':'categoria','Banos_Total':'baño_total','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
#if 'df_alfa_q_jul_2023_pachuca' in nombres_df_filtrados:
    #nombres_df_filtrados.remove('df_alfa_q_jul_2023_pachuca')

In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','antiguedad']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Meses de antigüedad

In [5]:
def categorizar_antiguedad_2023(valor):
    if valor == 7:
        return '7'
    elif valor == 8:
        return '8'
    elif valor == 9:
        return '9'
    elif valor == 10:
        return '10'
    elif valor == 11:
        return '11'
    elif valor == 12:
        return '12'
    elif valor == 13:
        return '13'
    elif valor == 14:
        return '14'
    elif valor == 15:
        return '15'
    elif valor == 16:
        return '1111'
    elif  17<= valor <= 18:
        return '16-18'
    elif 19 <= valor <= 24:
        return '19-24'
    elif valor >= 24:
        return '>24'
    else:
        return '11111'

for df in dataframes_list:
    df.loc[:, 'antiguedad_categoria'] = df['antiguedad'].apply(categorizar_antiguedad_2023)
    #df['antiguedad_categoria'] = df['antiguedad'].apply(categorizar_antiguedad_2023)
    print(df)

        id categoria  antiguedad antiguedad_categoria
0        1        B1           6                11111
1        2        S2           9                    9
2        3        B2          12                   12
3        4        E1          16                 1111
4        5        E1           9                    9
...    ...       ...         ...                  ...
9338  9339        EL           7                    7
9339  9340        EL          11                   11
9340  9341        EL           7                    7
9341  9342        S3          16                 1111
9342  9343        S2          16                 1111

[9343 rows x 4 columns]
          id categoria  antiguedad antiguedad_categoria
0          1        E1           0                11111
1          2        E1           1                11111
2          3        E1           0                11111
3          4        E1           0                11111
4          5        E1           9             

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\849075037.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'antiguedad_categoria'] = df['antiguedad'].apply(categorizar_antiguedad_2023)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\849075037.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'antiguedad_categoria'] = df['antiguedad'].apply(categorizar_antiguedad_2023)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\849075037.py:32: SettingWithCopyWarning: 
A value is trying to 

Conteo único de coincidencias con la antigüedad

In [6]:
# Orden personalizado
orden_personalizado = ['7', '8', '9', '10', '11', '12', '13','14','15','16-18','19-24','>24']

tabla_suma = None
# Iterar sobre cada DataFrame en la lista
for df in dataframes_list:
    # Crear una nueva columna con la primera letra de la categoría
    #df['segmento'] = df['categoria'].str[0]
    df.loc[:, 'segmento'] = df['categoria'].str[0]
    #df.drop(columns=['categoria'], inplace=True)        
    # Agrupar y contar los datos
    tabla_actual = df.groupby(['segmento', 'antiguedad_categoria'],observed=False).size().unstack(fill_value=0)
    
    # Sumar la tabla actual al acumulador (tabla_suma)
    if tabla_suma is None:
        tabla_suma = tabla_actual
    else:
        tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)

# Reordenar las columnas según el orden personalizado
tabla_suma = tabla_suma[orden_personalizado]

# Imprimir la tabla acumulativa
print(tabla_suma)

antiguedad_categoria       7      8       9     10     11     12     13  \
segmento                                                                  
A                      281.0  168.0   599.0  160.0  164.0  112.0  112.0   
B                      336.0  193.0   568.0  133.0  208.0  138.0  100.0   
C                      140.0   91.0   291.0  108.0   61.0   50.0   32.0   
D                      140.0  100.0   188.0   65.0   58.0   71.0   49.0   
E                      199.0  149.0   280.0   73.0  138.0   65.0   68.0   
L                      146.0   37.0   262.0   62.0   71.0   83.0   75.0   
S                     1044.0  736.0  2665.0  698.0  791.0  573.0  527.0   

antiguedad_categoria     14     15   16-18  19-24   >24  
segmento                                                 
A                      99.0  201.0   695.0  201.0   0.0  
B                      85.0  160.0   658.0  236.0   2.0  
C                      39.0   85.0   367.0  125.0   0.0  
D                      35.0   41.0

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\2385401544.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\2385401544.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\2385401544.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [7]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']
orden_personalizado = ['7', '8', '9', '10', '11', '12', '13','14','15','16-18','19-24','>24']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", ]

def sumar_tablas(dataframes_list):
    tabla_suma = None
    for df in dataframes_list:
        #df['segmento'] = df['categoria'].str[0]
        df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
        df.loc[:, 'segmento'] = df['categoria'].str[0]
        tabla_actual = df.groupby(['segmento', 'antiguedad_categoria'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    tabla_suma = tabla_suma[orden_personalizado]
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Para guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_tiempoventa', carpeta='assets/graficas')
    
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index  
    tiempo = tabla.columns
    
    fig = go.Figure()
        
    for i, tiempo_categoria in enumerate(tiempo):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=segmentos,
            y=tabla[tiempo_categoria],
            name=tiempo_categoria,
            marker_color=color
        ))
    
    fig.update_layout(
        #title="Meses de antigüedad",
        yaxis=dict(
            title="Número de casas",
            gridcolor='#dddcda',
        ),
        xaxis=dict(title="Segmentos"),
        legend_title="Antigüedad",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        barmode='group',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',
    )
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


# Supongamos que tienes una lista de DataFrames llamada dataframes_list
fig = sumar_tablas(dataframes_list)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\43125850.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\43125850.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_3808\43125850.py:14: SettingWithCopyWarning: 
A value is trying to

Tabla general de frecuencia:
antiguedad_categoria     7      8     9   10   11   12   13   14   15  16-18  \
segmento                                                                       
E                      199  149.0   280   73  138   65   68   19   46    112   
D                      140  100.0   188   65   58   71   49   35   41    149   
C                      140   91.0   291  108   61   50   32   39   85    367   
B                      336  193.0   568  133  208  138  100   85  160    658   
A                      281  168.0   599  160  164  112  112   99  201    695   
S                     1044  736.0  2665  698  791  573  527  402  583   2663   
L                      146   37.0   262   62   71   83   75   19   87    168   

antiguedad_categoria  19-24   >24  
segmento                           
E                        36   0.0  
D                        24   0.0  
C                       125   0.0  
B                       236   2.0  
A                       201   0.0 

## Totales

In [8]:
# Orden personalizado
orden_personalizado = ['7', '8', '9', '10', '11', '12', '13','14','15','16-18','19-24','>24']

tabla_suma = None
# Iterar sobre cada DataFrame en la lista
for df in dataframes_list:
    # Contar los registros por cada categoría de antigüedad
    tabla_actual = df['antiguedad_categoria'].value_counts().sort_index()    
    
    # Sumar la tabla actual al acumulador (tabla_suma)
    if tabla_suma is None:
        tabla_suma = tabla_actual
    else:
        tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)

# Reordenar las columnas según el orden personalizado
tabla_suma = tabla_suma[orden_personalizado]

# Crear un DataFrame a partir de la tabla acumulativa
df_acumulado = pd.DataFrame(tabla_suma)
df_acumulado.columns = ['count']
df_acumulado['antiguedad_categoria'] = df_acumulado.index

# Reiniciar el índice del DataFrame
df_acumulado.reset_index(drop=True, inplace=True)
df_acumulado

,count,antiguedad_categoria
0,2444.0,7
1,1574.0,8
2,5098.0,9
3,1413.0,10
4,1603.0,11
5,1152.0,12
6,1037.0,13
7,765.0,14
8,1275.0,15
9,5104.0,16-18


In [9]:
fig = go.Figure()
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059',]
count = df_acumulado['count']

fig.add_trace(go.Bar(
    x=df_acumulado['antiguedad_categoria'], 
    y=df_acumulado['count'], 
    marker_color=colores,  # Especifica los colores de las barras
    text=count,  # Texto que se mostrará en las barras 
    textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
    #texttemplate='%{text:.1f}%',  # Formato del texto (porcentaje con un decimal)
))
# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Meses de antigüedad',  # Título de la gráfica
   xaxis=dict(title='Meses de antigüedad'),  # Título del eje x
    yaxis=dict(
        title='Número de casas', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


# Exportar
guardar_grafico_como_html(fig, 'g_bar_tiempoventa_total', carpeta='assets/graficas')

fig.show()